<a href="https://colab.research.google.com/github/avxsharma-glitch/Numerical-optimization/blob/main/Linear_Algebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy.linalg import matrix_rank, det, solve, eig, norm

# ==========================================
# DATA SETUP
# ==========================================
# Matrix A: Interconnectedness between 4 banks
A = np.array([
    [ 2, -1,  0,  0],
    [-1,  2, -1,  0],
    [ 0, -1,  2, -1],
    [ 0,  0, -1,  2]
])

# Vector b: External Market Shocks
# ⚠️ ACTION: Replace these numbers with the correct values from your assignment
b = np.array([1, 2, 2, 1])

# Vector X: Initial Portfolio
# ⚠️ ACTION: Replace these numbers with the correct values from your assignment
X_portfolio = np.array([100, 100, 100, 100])

print("--- Data Loaded ---")
print(f"Matrix A:\n{A}")
print(f"Shock Vector b: {b}")

# ==========================================
# TASK 1: MATRIX ANALYSIS
# ==========================================
print("\n--- TASK 1: MATRIX PROPERTIES ---")

# Step 1: Symmetry Check
# A matrix is symmetric if A == A_transpose.
# Financial meaning: If Bank A affects Bank B, does Bank B affect Bank A equally?
is_symmetric = np.allclose(A, A.T)
print(f"Is the network symmetric? {is_symmetric}")

# Step 2: Key Properties
rank_A = matrix_rank(A)
det_A = det(A)
trace_A = np.trace(A)

print(f"Rank: {rank_A} (Full rank means all banks are uniquely significant)")
print(f"Determinant: {det_A:.2f} (Non-zero means the system is stable/invertible)")
print(f"Trace: {trace_A}")

# Step 3: Solve Ax = b
# This calculates the 'response' of the banks to the market shock 'b'
try:
    x_solution = solve(A, b)
    print(f"System Response (Solution to Ax=b):\n{x_solution}")
except np.linalg.LinAlgError:
    print("System has no unique solution (Singular Matrix).")

# ==========================================
# TASK 2: EIGEN ANALYSIS
# ==========================================
print("\n--- TASK 2: EIGENVALUES & VULNERABILITY ---")

# Step 1: Compute Eigenvalues & Eigenvectors
eigenvalues, eigenvectors = eig(A)

# Step 2: Identify Dominant (Largest) and Vulnerable (Smallest)
# Sort them to make it easier to identify
idx = eigenvalues.argsort()[::-1]
sorted_vals = eigenvalues[idx]
sorted_vecs = eigenvectors[:, idx]

dominant_val = sorted_vals[0]    # Largest eigenvalue
vulnerable_val = sorted_vals[-1] # Smallest eigenvalue

print(f"Dominant Eigenvalue (Max Volatility): {dominant_val:.4f}")
print(f"Vulnerable Eigenvalue (Min Volatility): {vulnerable_val:.4f}")

# Step 3: Cayley-Hamilton Validation
# Theorem: A matrix satisfies its own characteristic equation p(A) = 0
coeffs = np.poly(A)
n = A.shape[0]
check_matrix = np.zeros_like(A, dtype=float)

for i, c in enumerate(coeffs):
    power = n - i
    check_matrix += c * np.linalg.matrix_power(A, power)

# The result should be very close to a zero matrix
print("Cayley-Hamilton Check (Should be near zero):\n", np.round(check_matrix, 4))

# ==========================================
# TASK 3: PORTFOLIO RISK
# ==========================================
print("\n--- TASK 3: PORTFOLIO RISK ASSESSMENT ---")

# Step 1: Span Check
# Does our portfolio X lie within the "danger zone" (span of dominant eigenvectors)?
# We take the top 2 eigenvectors (v1, v2) and our portfolio X.
# If Rank([v1, v2, X]) is 2, then X is "trapped" in their span. If 3, it's outside.

v1 = sorted_vecs[:, 0]
v2 = sorted_vecs[:, 1]
span_matrix = np.column_stack((v1, v2, X_portfolio))
span_rank = matrix_rank(span_matrix)

if span_rank == 2:
    print("RISK ALERT: Portfolio lies within the span of dominant eigenvectors.")
else:
    print("SAFE: Portfolio lies OUTSIDE the span of dominant eigenvectors.")

# Step 2: Simulation
# How does the portfolio evolve? (A * X)
evolution = A.dot(X_portfolio)
print(f"Portfolio Evolution after shock:\n{evolution}")

--- Data Loaded ---
Matrix A:
[[ 2 -1  0  0]
 [-1  2 -1  0]
 [ 0 -1  2 -1]
 [ 0  0 -1  2]]
Shock Vector b: [1 2 2 1]

--- TASK 1: MATRIX PROPERTIES ---
Is the network symmetric? True
Rank: 4 (Full rank means all banks are uniquely significant)
Determinant: 5.00 (Non-zero means the system is stable/invertible)
Trace: 8
System Response (Solution to Ax=b):
[3. 5. 5. 3.]

--- TASK 2: EIGENVALUES & VULNERABILITY ---
Dominant Eigenvalue (Max Volatility): 3.6180
Vulnerable Eigenvalue (Min Volatility): 0.3820
Cayley-Hamilton Check (Should be near zero):
 [[ 0. -0.  0. -0.]
 [-0.  0. -0.  0.]
 [ 0. -0.  0. -0.]
 [-0.  0. -0.  0.]]

--- TASK 3: PORTFOLIO RISK ASSESSMENT ---
SAFE: Portfolio lies OUTSIDE the span of dominant eigenvectors.
Portfolio Evolution after shock:
[100   0   0 100]
